# RMSD drifters/DUACS – Verger-Miralles et al. (2025)

**SWOT enhances small-scale eddy detection in the Mediterranean Sea**

Author: *Elisabet Verger-Miralles*  
Institution: IMEDEA (CSIC-UIB)

Compute RMSD SVPB Drifters vels. module vs DUACS

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import glob
from scipy.interpolate import griddata
from datetime import datetime
from functions import *
from scipy.stats import bootstrap

In [ ]:
# 1. DRIFTERS
filedir = '../../grl_codes_to_publish_def_swotv2.0.1/data/drifters_filtered/SVPB/'
dd1 = '2023-04-23T00:00:00.000000000'
dd2 = '2023-04-29T00:00:00.000000000'
drifter_ids = ['035', '039', '040', '041', '042']
ds_drifters = []

for num in drifter_ids:
    url = filedir + f"drifter-svpb{num}_inertial_osc_filt_subset.nc"
    ds = xr.open_dataset(url).sel(time=slice(dd1, dd2))
    ds_drifters.append(ds)

# 2. DUACS
dir_DUACS = '../../grl_codes_to_publish_def_swotv2.0.1/data/DUACS/'
files_d = np.sort(glob.glob(dir_DUACS + 'dt*.nc'))[0:8]
lon_min, lon_max, lat_min, lat_max = 1.1, 1.9, 39.6, 40.1

df_DUACS = None
for file in files_d:
    ds_DUACS = xr.open_dataset(file)
    df_DUACS = process_and_concat_duacs(ds_DUACS, lon_min, lon_max, lat_min, lat_max, df_DUACS)

df_DUACS = df_DUACS.dropna().reset_index(drop=True)
df_DUACS['mean_time'] = df_DUACS.groupby(df_DUACS['time'].dt.date)['time'].transform('mean')

group_transect_ugos = df_DUACS.groupby('mean_time')['ugos'].apply(list).reset_index()
group_transect_vgos = df_DUACS.groupby('mean_time')['vgos'].apply(list).reset_index()
group_transect_lon = df_DUACS.groupby('mean_time')['longitude'].apply(list).reset_index()
group_transect_lat = df_DUACS.groupby('mean_time')['latitude'].apply(list).reset_index()

# Mean per transect
mean_transect_ugos = []
mean_transect_vgos = []
for i in range(len(group_transect_ugos)):
    int_u = griddata((np.array(group_transect_lon['longitude'][i]),
                      np.array(group_transect_lat['latitude'][i])),
                     np.array(group_transect_ugos['ugos'][i]),
                     (np.array(ds_drifters[0].LON), np.array(ds_drifters[0].LAT)), method='cubic')
    int_v = griddata((np.array(group_transect_lon['longitude'][i]),
                      np.array(group_transect_lat['latitude'][i])),
                     np.array(group_transect_vgos['vgos'][i]),
                     (np.array(ds_drifters[0].LON), np.array(ds_drifters[0].LAT)), method='cubic')
    mean_transect_ugos.append(np.nanmean(int_u))
    mean_transect_vgos.append(np.nanmean(int_v))

unique_mean_times = df_DUACS['mean_time'].unique()
df_DUACS['ugos_mean_transect'] = df_DUACS['mean_time'].map({t: mean_transect_ugos[i] for i, t in enumerate(unique_mean_times)})
df_DUACS['vgos_mean_transect'] = df_DUACS['mean_time'].map({t: mean_transect_vgos[i] for i, t in enumerate(unique_mean_times)})

# 3. SPATIOTEMPORAL INTERP. FOR ALL THE DRIFTERS
u_svp_all, v_svp_all, u_duacs_all, v_duacs_all = [], [], [], []

for ds in ds_drifters:
    Lon, Lat = ds.LON.values, ds.LAT.values
    u_svp, v_svp = ds.U.values, ds.V.values
    t = np.array(ds.time)

    u_duacs_int, v_duacs_int = [], []

    for i in range(len(Lon)):
        time_drifter = np.datetime64(t[i])
        lon_drifter, lat_drifter = Lon[i], Lat[i]

        df_DUACS['time_diff'] = abs(df_DUACS['mean_time'] - time_drifter)
        sorted_diffs = df_DUACS['time_diff'].drop_duplicates().sort_values()
        if len(sorted_diffs) < 2: continue
        min1, min2 = sorted_diffs.iloc[0], sorted_diffs.iloc[1]

        for min_diff in [min1, min2]:
            subset = df_DUACS[df_DUACS['time_diff'] == min_diff]
            ugos_an = subset['ugos'].values - np.nanmean(subset['ugos_mean_transect'].values)
            vgos_an = subset['vgos'].values - np.nanmean(subset['vgos_mean_transect'].values)
            lon_sw, lat_sw = subset['longitude'].values, subset['latitude'].values

            u_interp = griddata((lon_sw, lat_sw), ugos_an, (lon_drifter, lat_drifter), method='cubic')
            v_interp = griddata((lon_sw, lat_sw), vgos_an, (lon_drifter, lat_drifter), method='cubic')

            if min_diff == min1:
                u_interp1, v_interp1 = u_interp, v_interp
            else:
                u_interp2, v_interp2 = u_interp, v_interp

        denom = min1 + min2
        u_int_def = (u_interp1 * min2 + u_interp2 * min1) / denom
        v_int_def = (v_interp1 * min2 + v_interp2 * min1) / denom

        u_duacs_int.append(u_int_def)
        v_duacs_int.append(v_int_def)

    u_svp_all.extend(u_svp)
    v_svp_all.extend(v_svp)
    u_duacs_all.extend(u_duacs_int)
    v_duacs_all.extend(v_duacs_int)

u_svp_all = np.array(u_svp_all)
v_svp_all = np.array(v_svp_all)
u_duacs_all = np.array(u_duacs_all)
v_duacs_all = np.array(v_duacs_all)

# Module
abs_vel_svp = np.sqrt(u_svp_all**2 + v_svp_all**2)
abs_vel_duacs = np.sqrt(u_duacs_all**2 + v_duacs_all**2)

rmsd = np.sqrt(np.nanmean((abs_vel_svp - abs_vel_duacs) ** 2))
print(f'RMSE (módulo): {rmsd:.4f} m/s')

# # Dirección
# dir_vel_svp = np.degrees(np.arctan2(u_svp_all, v_svp_all))
# dir_vel_duacs = np.degrees(np.arctan2(u_duacs_all, v_duacs_all))

# def ang_err_180(angles):
#     angles = np.where(angles < -180., 360 + angles, angles)
#     angles = np.where(angles > 180., angles - 360, angles)
#     return angles

# angle_diff = ang_err_180(dir_vel_svp - dir_vel_duacs)
# rmse_dir = np.sqrt(np.nanmean(angle_diff ** 2))
# print(f'RMSE dirección: {rmse_dir:.2f} grados')


RMSE (módulo): 0.1076 m/s
RMSE dirección: 30.63 grados


## BOOTSTRAP

MODULE

In [3]:
def rmsd(drifter, swot, axis=0):
    """Compute Root Mean Square Deviation (RMSD)."""
    diff = drifter - swot
    return np.sqrt(np.nanmean(diff**2, axis=axis))

# Combine data into a tuple without reshaping
data = (abs_vel_svp, abs_vel_duacs)

# Perform bootstrap resampling
res = bootstrap(
    data, 
    statistic=rmsd, 
    n_resamples=1000, 
    confidence_level=0.95, 
    method='BCa',  # Bias-Corrected and Accelerated bootstrap method
    paired=True,  # Since we compare paired velocity values
    random_state=42  # For reproducibility
)

# Print results
print(f"RMSD: {rmsd(abs_vel_svp, abs_vel_duacs):.4f}")
# print(f"RMSD: {rmsd(np.array(subsampling), np.array(subsampling_duacs)):.4f}")

print(f"95% Confidence Interval: {res.confidence_interval}")

RMSD: 0.1076
95% Confidence Interval: ConfidenceInterval(low=np.float64(0.10443535539969981), high=np.float64(0.11096026330652639))


In [4]:
# low confidence interval
low_ci = res.confidence_interval[0]
high_ci = res.confidence_interval[1]
low_ci, high_ci

(np.float64(0.10443535539969981), np.float64(0.11096026330652639))

In [5]:
0.1076 - low_ci, high_ci - 0.1076 # longitude of the error bar

(np.float64(0.0031646446003001927), np.float64(0.00336026330652639))

In [6]:
(((0.1076 - low_ci)*100) + ((high_ci - 0.1076)*100))/2

np.float64(0.3262453953413291)